# WANN experiments on Sentiment Analysis dataset

In [1]:
import copy
import sys

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.constraints import MinMaxNorm
from tensorflow.keras.optimizers import Adam

sys.path.append("../wann")
sys.path.append("../wann/methods")
from utils import sa
from WANN import WANN

from adapt.instance_based import KLIEP, KMM, TrAdaBoostR2
from adapt.feature_based import MDD, DANN, ADDA, DeepCORAL

from warnings import filterwarnings
filterwarnings('ignore')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Base Estimator

In [2]:
def get_base_model(shape=1000, activation=None, C=1, name="BaseModel"):
    inputs = Input(shape=(shape,))
    modeled = Dense(100, activation='relu',
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(inputs)
    modeled = Dropout(0.8)(modeled)
    modeled = Dense(100, activation='relu',
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(modeled)
    modeled = Dropout(0.5)(modeled)
    modeled = Dense(1, activation=activation,
                    kernel_constraint=MinMaxNorm(0, C),
                    bias_constraint=MinMaxNorm(0, C))(modeled)
    model = Model(inputs, modeled, name=name)
    model.compile(optimizer=Adam(0.001), loss='mean_squared_error')
    return model


def get_encoder(shape=1000, C=1, name="encoder"):
    inputs = Input(shape=(shape,))
    modeled = Dense(100, activation='relu',
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(inputs)
    modeled = Dropout(0.8)(modeled)
    model = Model(inputs, modeled)
    model.compile(optimizer=Adam(0.001), loss='mean_squared_error')
    return model


def get_task(shape=100, C=1, activation=None, name="task"):
    inputs = Input(shape=(shape,))
    modeled = Dense(100, activation='relu',
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(inputs)
    modeled = Dropout(0.5)(modeled)
    modeled = Dense(1, activation=activation,
                         kernel_constraint=MinMaxNorm(0, C),
                         bias_constraint=MinMaxNorm(0, C))(modeled)
    model = Model(inputs, modeled)
    model.compile(optimizer=Adam(0.001), loss='mean_squared_error')
    return model

In [4]:
np.random.seed(0)
seeds = np.random.choice(2**16, 10)

epochs = 200
batch_size = 128

N = 50   # Number of labeled target data
m = 700  # Number of labeled source data
n = 700  # Number of unlabeled target data


for source in ["books", "electronics", "kitchen", "dvd"]:
    for target in ["books", "electronics", "kitchen", "dvd"]:
        if source != target:
            
            print(source, target)
            
            scores = {}
            scores_cv = {}

            for i in [2,3,4,5,6,7,8,9,1,0]:

                np.random.seed(seeds[i])
                tf.random.set_seed(seeds[i])
                
                X, y, src_index, tgt_index = sa(source, target)
                mu = 3.; std = 1.5
                y = (y - 3.) / 1.5
                shape = X.shape[1]

                src_index = np.random.choice(src_index, m, replace=False)
                tgt_index, tgt_test_index = train_test_split(tgt_index, train_size=n, test_size=1000)
                tgt_index_labeled = np.random.choice(tgt_index, N, replace=False)
                train_index = np.concatenate((src_index, tgt_index_labeled))
                
                test_ = np.random.choice(tgt_index_labeled, 10, replace=False)
                train_ = np.array(list(set(tgt_index_labeled) - set(test_)))
                
                train_index_cv = np.concatenate((src_index, train_))

                for method in ["WANN", "TgtOnly", "NoReweight", "KLIEP", "KMM", "DANN", "ADDA", "DeepCORAL", "MDD", "TrAdaBoostR2"]:
                    print(method)
                    if not method in scores:
                        scores[method] = []
                        if method in ["WANN", "KLIEP", "KMM", "DANN", "DeepCORAL", "MDD"]:
                            scores_cv[method] = []
                    
                    cv_scores = []
                    
                    if method in ["TgtOnly", "NoReweight"]:
                        model = get_base_model(shape=X.shape[1])

                        if method == "TgtOnly":
                            model.fit(X[tgt_index_labeled], y[tgt_index_labeled], epochs=epochs, batch_size=batch_size, verbose=0)
                        if method == "NoReweight":
                            model.fit(X[train_index], y[train_index], epochs=epochs, batch_size=batch_size, verbose=0)

                    elif method in ["WANN"]:
                        for C_w in [0.1, 0.2, 0.5, 1.]:
                            model = WANN(get_base_model=get_base_model, C=1., C_w=C_w, optimizer=Adam(0.001))   
                            model.fit(X, y, [train_index, train_], epochs=epochs, verbose=0, batch_size=batch_size)
                            err_t = np.mean(np.square(model.predict(X).ravel()[test_] - y[test_]))
                            print("%.3f, target %.3f"%(C_w, err_t))
                            cv_scores.append(err_t)
                        scores_cv[method].append(cv_scores)
                        arg = np.argmin(cv_scores)
                        C_w = [0.1, 0.2, 0.5, 1.][arg]
                        model = WANN(get_base_model=get_base_model, C=1., C_w=C_w, optimizer=Adam(0.001))
                        model.fit(X, y, [train_index, tgt_index_labeled], epochs=epochs, verbose=0, batch_size=batch_size)
                        
                    elif method == "TrAdaBoostR2":
                        model = TrAdaBoostR2(get_base_model(), verbose=2, random_state=seeds[i])
                        model.fit(X[train_index], y[train_index], X[tgt_index_labeled], y[tgt_index_labeled],
                                  epochs=epochs, batch_size=batch_size, verbose=0)

                    else:            
                        if method == "DANN":
                            for lambda_ in [0.0001, 0.001, 0.01, 0.1, 1.]:
                                model = DANN(encoder=get_encoder(), task=get_task(), random_state=seeds[i],
                                             discriminator=get_task(activation="sigmoid"),
                                             optimizer=Adam(0.001), lambda_=lambda_, loss="mse")
                                model.fit(X[train_index], y[train_index], X[tgt_index], epochs=epochs, batch_size=batch_size, verbose=0)
                                err_t = np.mean(np.square(model.predict(X).ravel()[test_] - y[test_]))
                                print("%.3f, target %.3f"%(lambda_, err_t))
                                cv_scores.append(err_t)
                            scores_cv[method].append(cv_scores)
                            arg = np.argmin(cv_scores)
                            lambda_ = [0.0001, 0.001, 0.01, 0.1, 1.][arg]
                            model = DANN(encoder=get_encoder(), task=get_task(), random_state=seeds[i],
                                             discriminator=get_task(activation="sigmoid"),
                                             optimizer=Adam(0.001), lambda_=lambda_, loss="mse")
                        if method == "DeepCORAL":
                            for lambda_ in [0.1, 1., 10, 100., 1000.]:
                                model = DeepCORAL(encoder=get_encoder(), task=get_task(), lambda_=lambda_,
                                                  optimizer=Adam(0.001), loss="mse",
                                                  random_state=seeds[i])
                                model.fit(X[train_index], y[train_index], X[tgt_index], epochs=epochs, batch_size=batch_size, verbose=0)
                                err_t = np.mean(np.square(model.predict(X).ravel()[test_] - y[test_]))
                                print("%.3f, target %.3f"%(lambda_, err_t))
                                cv_scores.append(err_t)
                            scores_cv[method].append(cv_scores)
                            arg = np.argmin(cv_scores)
                            lambda_ = [0.1, 1., 10, 100., 1000.][arg]
                            model = DeepCORAL(encoder=get_encoder(), task=get_task(), lambda_=lambda_,
                                                  optimizer=Adam(0.001), loss="mse",
                                                  random_state=seeds[i])
                        if method == "MDD":
                            for lambda_ in [0.0001, 0.001, 0.01, 0.1, 1.]:
                                model = MDD(encoder=get_encoder(), task=get_task(), random_state=seeds[i],
                                            optimizer=Adam(0.001), lambda_=lambda_, loss="mse")
                                model.fit(X[train_index], y[train_index], X[tgt_index], epochs=epochs, batch_size=batch_size, verbose=0)
                                err_t = np.mean(np.square(model.predict(X).ravel()[test_] - y[test_]))
                                print("%.3f, target %.3f"%(lambda_, err_t))
                                cv_scores.append(err_t)
                            scores_cv[method].append(cv_scores)
                            arg = np.argmin(cv_scores)
                            lambda_ = [0.0001, 0.001, 0.01, 0.1, 1.][arg]
                            model = MDD(encoder=get_encoder(), task=get_task(), random_state=seeds[i],
                                            optimizer=Adam(0.001), lambda_=lambda_, loss="mse")
                        if method == "ADDA":
                            encoder = get_encoder()
                            task=get_task()
                            discriminator=get_task(activation="sigmoid")
                            dann = DANN(encoder, task, discriminator, loss="mse", copy=False,
                                        lambda_=0., random_state=seeds[i])
                            dann.fit(X[train_index], y[train_index], X[tgt_index], epochs=epochs, batch_size=batch_size, verbose=0)
                            model = ADDA(encoder=encoder, task=task,
                                         discriminator=discriminator, random_state=seeds[i],
                                         is_pretrained=True,
                                         optimizer=Adam(0.001), loss="mse")
                        if method == "KLIEP":
                            for sigmas in [0.0001, 0.001, 0.01, 0.1, 1.]:
                                model = KLIEP(get_base_model(), sigmas=sigmas, random_state=seeds[i])
                                model.fit(X[train_index], y[train_index], X[tgt_index], epochs=epochs, batch_size=batch_size, verbose=0)
                                err_t = np.mean(np.square(model.predict(X).ravel()[test_] - y[test_]))
                                print("%.3f, target %.3f"%(sigmas, err_t))
                                cv_scores.append(err_t)
                            scores_cv[method].append(cv_scores)
                            arg = np.argmin(cv_scores)
                            sigmas = [0.0001, 0.001, 0.01, 0.1, 1.][arg]
                            model = KLIEP(get_base_model(), sigmas=sigmas, random_state=seeds[i])
                        if method == "KMM":
                            for sigmas in [0.0001, 0.001, 0.01, 0.1, 1.]:
                                model = KMM(get_base_model(), kernel_params=dict(gamma=sigmas), verbose=0, random_state=seeds[i])
                                model.fit(X[train_index], y[train_index], X[tgt_index], epochs=epochs, batch_size=batch_size, verbose=0)
                                err_t = np.mean(np.square(model.predict(X).ravel()[test_] - y[test_]))
                                print("%.3f, target %.3f"%(sigmas, err_t))
                                cv_scores.append(err_t)
                            scores_cv[method].append(cv_scores)
                            arg = np.argmin(cv_scores)
                            sigmas = [0.0001, 0.001, 0.01, 0.1, 1.][arg]
                            model = KMM(get_base_model(), kernel_params=dict(gamma=sigmas), verbose=0, random_state=seeds[i])
                            
                        model.fit(X[train_index], y[train_index], X[tgt_index], epochs=epochs, batch_size=batch_size, verbose=0)
                    
                    err_s = np.mean(np.square(model.predict(X).ravel()[src_index] - y[src_index]))
                    err_t = np.mean(np.square(model.predict(X).ravel()[tgt_test_index] - y[tgt_test_index]))

                    scores[method].append(err_t)

                    print("source %.3f"%err_s)
                    print("target %.3f"%err_t)


                pd.DataFrame(scores_cv).to_csv("../dataset/results/amazon_cv_%s%s.csv"%(source, target))
                pd.DataFrame(scores).to_csv("../dataset/results/amazon_goodparams_%s%s.csv"%(source, target))